In [1]:
import json

def hard_voting(file_list):
    # 모든 파일의 데이터를 저장할 리스트
    all_data = []

    # 각 파일에서 데이터 로드
    for file in file_list:
        with open(file, 'r') as f:
            data = [json.loads(line) for line in f.readlines()]
            all_data.append(data)

    final_results = []

    # 각 id에 대해 투표 시작
    for i in range(len(all_data[0])):
        vote_result = {"joy": 0, "anticipation": 0, "trust": 0, "surprise": 0,
                       "disgust": 0, "fear": 0, "anger": 0, "sadness": 0}

        # 모든 파일의 예측 결과에 대해 투표 진행
        for data in all_data:
            for emotion in vote_result.keys():
                if data[i]['output'][emotion] == 'True':
                    vote_result[emotion] += 1

        max_vote = max(vote_result.values())

        if max_vote == 0:   # 만약 모든 감정이 False라면 
            output_dict = {emotion: 'True' if emotion == 'joy' else 'False'
                           for emotion in vote_result.keys()}
        else:
            output_dict = {emotion: 'True' if vote == max_vote else 'False'
                           for emotion, vote in vote_result.items()}

        final_results.append({"id" : all_data[0][i]['id'], 
                              'input': all_data[0][i]['input'], 
                              'output': output_dict})
        
    ensemble_filename = "+".join([file[len(file_path):].split('.')[0] for file in file_list])
    
    with open(f"{file_path}ensemble({ensemble_filename}).jsonl", mode='w') as outfile:
        for entry in final_results:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')

    return final_results


# 사용 예시
file_path = '../outputs/'
# file_list = ['32.jsonl', '11.jsonl', '5.jsonl', '17.jsonl'] # # foundation 모델이 같아도 전처리 기법이 다른 top-4 모델
# file_list = ['32.jsonl', '5.jsonl', '17.jsonl'] # foundation 모델이 서로 다른 top-3 모델
# file_list = ['32.jsonl', '25.jsonl', '31.jsonl', '34.jsonl', '26.jsonl', '30.jsonl', '35.jsonl', '28.jsonl'] # public test score top-8 모델
# file_list = ['32.jsonl', '25.jsonl', '31.jsonl', '34.jsonl', '26.jsonl'] # public test score top-5 모델
file_list = ['32.jsonl', '25.jsonl', '31.jsonl'] # public test score top-5 모델
file_list = [file_path + file_name for file_name in file_list]
final_results = hard_voting(file_list)